In [ ]:
!pip install mediapipe
!pip install pafy
!pip install youtube-dl==2020.12.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 31.5 MB 1.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 5.1 MB/s 


In [ ]:
!pip install opencv-contrib-python==4.5.5.62

In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import matplotlib.pyplot as plt
import pafy
from mediapipe.python.solutions.drawing_utils import _normalized_to_pixel_coordinates

In [ ]:
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5)

In [ ]:
def blur_image(image, factor=3.0):
	# automatically determine the size of the blurring kernel based
	# on the spatial dimensions of the input image
	(h, w) = image.shape[:2]
	kW = int(w / factor)
	kH = int(h / factor)
	# ensure the width of the kernel is odd
	if kW % 2 == 0:
		kW -= 1
	# ensure the height of the kernel is odd
	if kH % 2 == 0:
		kH -= 1
	# apply a Gaussian blur to the input image using our computed
	# kernel size
	return cv2.GaussianBlur(image, (kW, kH), 0)

In [ ]:
def Final_Solution(url):
  video = pafy.new(url)
  best  = video.getbest()
  cap = cv2.VideoCapture(best.url)
  frame_width = int(cap.get(3))
  frame_height = int(cap.get(4))
  out = cv2.VideoWriter('output.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))
  while(True):
    ret, frame = cap.read()
    if ret:
      img=frame.copy()
      image_rows, image_cols, _ = img.shape
      face_detection_results = face_detection.process(img)
      detection=face_detection_results.detections
      #if face is detected or not
      if detection != None:
        face_data = detection[0].location_data
        relative_bounding_box = face_data.relative_bounding_box
        xmin=relative_bounding_box.xmin
        ymin=relative_bounding_box.ymin
        width=relative_bounding_box.width
        height=relative_bounding_box.height
        rect_start_point = _normalized_to_pixel_coordinates(xmin,ymin , image_cols, image_rows)
        rect_end_point = _normalized_to_pixel_coordinates(xmin + width,ymin + height , image_cols,image_rows)
        #cv2.rectangle(img, rect_start_point,rect_end_point, (255, 0, 0), 2)
        #handle when the limit of rect end point is over frame size
        if rect_end_point==None or rect_start_point==None:
          continue
        xleft,ytop=rect_start_point
        xright,ybot=rect_end_point
        crop_img = img[ytop: ybot, xleft: xright]
        blur_img=blur_image(crop_img)
        img[ytop: ybot, xleft: xright]=blur_img
        out.write(img)
    
      else:
        pass
    else:
      break
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
      break



  cap.release()
  out.release()
  cv2.destroyAllWindows

In [ ]:
url = "https://www.youtube.com/watch?v=F9aveOkw0yA"
url = "https://www.youtube.com/watch?v=1j0X9QMF--M"
Final_Solution(url)
